In [1]:
from manim import *
import numpy as np

In [2]:
grid_size = 0.01
grid_point_count = int(1 / grid_size)
single_axis = np.linspace(0, 1 - grid_size, grid_point_count)
M = np.asarray(np.meshgrid(single_axis, single_axis, single_axis, indexing='ij'))
X, Y, Z = M
M.shape

(3, 100, 100, 100)

In [3]:
flattened_M = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
flattened_M

array([[0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.01],
       [0.  , 0.  , 0.02],
       ...,
       [0.99, 0.99, 0.97],
       [0.99, 0.99, 0.98],
       [0.99, 0.99, 0.99]], shape=(1000000, 3))

In [4]:
def curve_function(t_values) -> np.array:
    base_point = np.array([0.5, 0.5, 0.5])
    return base_point + 0.4 * np.asarray([np.cos(2 * np.pi * t_values), np.sin(2 * np.pi * t_values), np.zeros_like(t_values)]).T # simple circle in xy plane.

In [5]:
t_values = np.linspace(0, 1, grid_point_count * 1)
simple_curve = curve_function(t_values)
simple_curve.shape

(100, 3)

In [6]:
# Dirac delta vector field form
def curve_tangent_function(t_values) -> np.array:
    # TODO: Implement numerical differentiation of curve function:
    return np.asarray([-np.sin(2*np.pi * t_values), np.cos(2 * np.pi * t_values), np.zeros_like(t_values)]).T

In [7]:
def numerical_tangent_function(t_values, curve_tangent_function, h = 0.001):
    """
    Compute the 4th-order central difference approximation of the derivative of f at t.

    Parameters:
    - f: function, the function to differentiate
    - t: float, the point at which to compute the derivative
    - h: float, the step size

    Returns:
    - float, the approximate derivative at t
    """
    
    def compute_4th_order_approximation(t):
        f = curve_tangent_function
        return (1 / (12 * h)) * (-f(t + 2 * h) + 8 * f(t + h) - 8 * f(t - h) + f(t - 2*h))
    
    return np.array([compute_4th_order_approximation(t) for t in t_values])

In [8]:
simple_curve_tangent = numerical_tangent_function(t_values, curve_function)

# Dirac-delta representation of a curve

## Initial thought

As the original authors defined, we can create a representation of a curve $\Gamma$ by considering a two form $\omega \in \Omega^2(M)$.
$\omega = g_1 dx^2 \wedge dx^3 + g_2 dx^3 \wedge dx^1 + g_3 dx^1 \wedge dx^2$.

The coefficients $g_1, g_2, g_3$ tell the direction of the curve. For example, a curve $\gamma : [0, 1] \to \mathbb{R}^3, \gamma(t) = (\cos(2\pi t), \sin(2\pi t), 0)$ can be represented as the 2-form

$$
\omega = \cos(2\pi t) dx^2 \wedge dx^3 + \sin(2\pi t) dx^3 \wedge dx^1 + 0 dx^1 \wedge dx^2
$$

This comes mathematically from the notion that we can identify each 2-form with an isomorphism to the space of vector fields, $(\star\omega)^\# = g_i \mathbf{e}^i$ where vectors $\mathbf{e}^i$ form the standard orthonormal basis for $M = \mathbb{R}^3$.

### Note on notation:

We use the Einstein summation convention, for example in the musical isomorphism we have a lower and upper index $i$ which signals that we want to sum those elements. Usually we try to keep the upper indices to be associated with vectors and differential forms, as opposed to lower indices with functions.


## $\delta_\Gamma$ form

The initial thought was a good start but does not capture our curve $\Gamma$ explicitly since there are multiple different curves which could have the same differential form representation (for the simple circle, think about extending the radius by 1).

We can fix this by augmenting the form $\omega$ to have infinite "magnitude" on the curve $\Gamma$ and $0$ everywhere else. Formally, we consider the Dirac-delta form $\delta_\Gamma$ to be the associated 2-form to a sequence of vector fields. The vector fields are defined as

$$
\mathbf{v}_\Gamma^\epsilon(\mathbf{x}) = \begin{cases}
\frac{1}{\epsilon^2 \pi} \mathbf{T}_\Gamma (\mathbf{x}^*), \text{ if } |x - x^*| \leq \epsilon, x^* = \arg \min_{p \in \Sigma}|x - p|\\
0, \text{ otherwise}
\end{cases}
$$

and the dirac-delta form $\delta_\Gamma$ is the limit of these vector fields.  
The limit is defined in the sense that for any 1-form $\eta \in \Omega(M)$ and its associated vector field $\eta^\#$ the line integral 

$$
\int_M \eta^\# \cdot \mathbf{v}_\Gamma^\epsilon dV
$$ 

approaches the integral of the 1-form on the curve $\Gamma$, namely $\int_\Gamma \eta$
, as $\epsilon \to 0$.

In [9]:
def dirac_delta_vector_field(x: np.array, distance_epsilon: float, curve_values, curve_tangent_values) -> np.array:
    # Compute the distance to input point
    # For every point on curve and every point in M, compute the L2 squared distance
    distances = []
    for point in curve_values:
        l2_squared = np.sum((x - point) ** 2, axis = 1)
        distances.append(l2_squared)
    distances = np.asarray(distances)

    # Get the minimum distances, for each point in M, obtain closest point p in curve Gamma:
    min_distance_indices = np.argmin(distances, axis = 0)
    minimum_points = curve_values[min_distance_indices, :]

    # Tangent vector points:
    tangent_vector_locations = curve_tangent_values[min_distance_indices]

    # Determine if the minimum distance x - x^* is less than epsilon:
    should_place_vector = np.asarray(np.linalg.norm(x - minimum_points, axis = 1) <= distance_epsilon)

    # Create the final vector field by placing tangent vectors at the appropriate points:
    curve_vector_field = (should_place_vector * np.ones((x.shape[1], 1))).T * tangent_vector_locations
    return curve_vector_field

In [10]:
#flattened_M = np.asarray([[1, 0, 0], [0, 1, 0], [np.cos(np.pi / 4), np.sin(np.pi / 4), 0], [0, 0, 0], [1, 1, 0]])
dirac_delta_epsilon = grid_size * 8
circle_vector_field = dirac_delta_vector_field(flattened_M, dirac_delta_epsilon, simple_curve, simple_curve_tangent)
print(circle_vector_field)


[[ 0. -0.  0.]
 [ 0. -0.  0.]
 [ 0. -0.  0.]
 ...
 [-0.  0.  0.]
 [-0.  0.  0.]
 [-0.  0.  0.]]


In [11]:
# nonzero_vector_indices = np.where(circle_vector_field != 0)[0]
# for p in nonzero_vector_indices:
#     print(f"{flattened_M[p]} : {circle_vector_field[p]}")

In [12]:
%%manim -v WARNING -ql --disable_caching VectorField3D
class VectorField3D(ThreeDScene):
    def construct(self):
        # Set up the axes
        axes = ThreeDAxes(
            x_range=[-2, 2, 0.2],
            y_range=[-2, 2, 0.2],
            z_range=[-2, 2, 0.2],
        )
        self.add(axes)

        # Define the vector field function
        def vector_field_func(point):
            # Find closest point
            distances_to_point = np.sum((flattened_M - point) ** 2, axis = 1)
            closest_point = np.argmin(distances_to_point)
            tangent_vector = circle_vector_field[closest_point]
            norm = np.linalg.norm(tangent_vector)
            normalized_vector = (tangent_vector / (norm + 1E-9)) * 0.4
            return normalized_vector

        vf = ArrowVectorField(vector_field_func, x_range=[0, 1, 0.05], y_range=[0, 1, 0.05], z_range=[0, 1, 0.05])
        vf.fit_to_coordinate_system(axes)
        self.add(vf)

        # Set up 3D camera rotation
        self.set_camera_orientation(phi=75 * DEGREES)
        self.wait(2)
        self.begin_ambient_camera_rotation(rate=0.1)
        self.wait(10)

Manim Community v0.18.1